# Complete guide

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt



import trecs
from trecs.models import ContentFiltering
from trecs.random import Generator
from trecs.metrics import MSEMeasurement, RecSimilarity, InteractionSimilarity, RecallMeasurement
from trecs.components import Users
import trecs.matrix_ops as mo #Note, in order for the ideal recommender to result in non-insane results, the normalize_users parameter in mo.inner_product must be set to False

In [2]:
class IdealRecommender(ContentFiltering):
    """
    With the Ideal Recommender, we make the *strong assumption* that the true scores are provided
    to the recommender system through a custom scoring function, which always returns the true
    underlying user-item scores. Therefore, this class is pretty much an empty skeleton; the only
    modification is that we don't update any internal state of the recommender at each time step.
    
    Amy: Note to self, as a part of the baseline ContentFiltering class, if an actual_item_representation 
    """
    def __init__(self, *args, **kwargs):
        
        
        super().__init__(*args, **kwargs)
    

    def _update_internal_state(self, interactions):
        # do not change users_hat!
        pass


In [ ]:
#np.random.seed(1)
plt.style.use('seaborn')
plt.rcParams.update({'font.size': 50})

In [3]:
# Keep the dimensions small for easy visualization
number_of_users = 5
number_of_attributes = 10
number_of_items = 15
# We define user_representation using the standard integer generator in Numpy.
# We assume a number of interactions with each attribute in the interval [0,4).

users_shape = (number_of_users, number_of_attributes)
actual_user_representation = Users(size=users_shape, num_users=number_of_users)
model_user_representation = np.random.randint(4, size=(number_of_users, number_of_attributes))

# We define item_representation using the Generator that comes with the framework
# We assume a binary matrix with a binomial distribution

actual_item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)

model_item_representation = Generator().binomial(
    n=1, p=.3, size=(number_of_attributes, number_of_items)
)

In [ ]:
#Not positive this is correct

content = ContentFiltering(
    num_users=number_of_users,
    num_items=number_of_items,
    num_attributes=number_of_attributes,
    actual_user_representation=actual_user_representation,
    user_representation=model_user_representation, 
    actual_item_representation = actual_item_representation,
    item_representation=actual_item_representation, #model has the true item values
)
# add an MSE measurement
content.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
content.run(timesteps=5)

In [6]:
ideal = IdealRecommender(
    num_users=number_of_users,
    num_items=number_of_items,
    num_attributes=number_of_attributes,
    actual_user_representation=actual_user_representation,
    user_representation=actual_user_representation.actual_user_profiles.value, 
    actual_item_representation = actual_item_representation,
    item_representation=actual_item_representation, #model has the true item values
    num_items_per_iter=10,
)
# add an MSE measurement
ideal.add_metrics(MSEMeasurement(), RecallMeasurement())
# Run for 5 time steps
ideal.run(timesteps=5)

100%|██████████| 5/5 [00:00<00:00, 1792.74it/s]


In [ ]:
ideal.actual_item_attributes == ideal.predicted_item_attributes

In [ ]:
ideal.actual_user_profiles == ideal.users_hat.value

In [ ]:
print(ideal.items_hat.value.shape)
print(ideal.users_hat.value.shape)

test=np.dot(ideal.users_hat.value,ideal.items_hat.value)
print(test.shape)
print(test)

In [ ]:
print(ideal.users.actual_user_scores.value)

In [5]:
dot = np.dot(ideal.users_hat.value,ideal.items_hat.value)
print(ideal.predicted_scores.shape)
print(ideal.users.actual_user_scores.shape)
print(dot.shape)

#Why aren't these three entities the same?

print(ideal.predicted_scores.value)
print('\n')
print(ideal.users.actual_user_scores.value)
print('\n')

print(np.dot(ideal.users_hat.value,ideal.items_hat.value))



(5, 15)
(5, 15)
(5, 15)
[[ 0.          1.26516938  1.65045296 -0.27550128  1.92011014 -1.42721844
   0.          0.89672002 -3.16803989 -0.52643267 -1.27202857 -0.49438955
  -0.79443041 -0.8726212  -0.32773772]
 [ 0.         -0.84487896 -3.06453793 -0.13639979 -4.92263048 -0.02228964
   0.         -0.14431139 -0.66294351 -0.71648374 -1.5545492  -3.879899
  -2.19947617 -3.8867134  -1.98648869]
 [ 0.          0.44593296  0.16793296  2.14316519 -0.11802906  1.88641389
   0.          0.98297234 -0.10660209 -0.23419789  1.05498935  0.06457004
  -0.70215349 -2.26259284 -1.08973977]
 [ 0.          0.64241915  1.12430541 -1.56262308  2.36549033 -3.1283265
   0.         -0.25054696 -2.72810574  0.56084514 -2.62257545 -2.82095302
  -0.11202093  2.51050699  2.82837921]
 [ 0.          1.80472856 -3.15938995 -0.43411372 -3.85697692  2.60233285
   0.         -1.35406446  2.07142817 -0.74627524 -1.58170818  1.09215876
   1.23269782  1.15022388 -0.72867956]]


[[ 0.          1.26516938  1.65045296 -0.

In [ ]:
#These are all the same in ideal as of 4/21 at 7PM
print(ideal.predicted_user_profiles)
print('\n')
print(ideal.users_hat.value)
print('\n')
print(ideal.actual_user_profiles)


In [ ]:
#These are all the same in ideal as of 4/21 at 7PM
print(ideal.predicted_item_attributes)
print('\n')
print(ideal.items_hat.value)
print('\n')

print(ideal.actual_item_attributes)

In [ ]:
#in recommender: 
# predicted_scores = self.score_fn(
#             self.predicted_user_profiles, self.predicted_item_attributes
#         )


###
#     def __init__(  # pylint: disable=R0913,R0912,R0915
#         self,
#         users_hat,
#         items_hat,
#         users,
#         items,
#         num_users,
#         num_items,
#         num_items_per_iter,
#         creators=None,
#         probabilistic_recommendations=False,
#         measurements=None,
#         record_base_state=False,
#         system_state=None,
#         score_fn=mo.inner_product,
#         interleaving_fn=None,
#         verbose=False,
#         seed=None,
#     ):

In [7]:
# Collect measurements about the simulation
results = ideal.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

Results of the simulation:


,mse,recall_at_k,timesteps
0,0.0,NaN,0
1,0.0,0.8,1
2,0.0,0.8,2
3,0.0,0.8,3
4,0.0,0.8,4
5,0.0,0.8,5


In [ ]:
results = content.get_measurements()

print("Results of the simulation:")
pd.DataFrame(results)

In [ ]:
#generate a distribution that hides a subpopulation
number_of_attributes = 10
number_of_maj_users = 150
number_of_min_users = 50

maj_user_representation = np.random.normal(1, 2, size=(number_of_maj_users, number_of_attributes))
min_user_representation = np.random.normal(0.5, 1.25, size=(number_of_min_users, number_of_attributes))
actual_user_representation = np.vstack((maj_user_representation, min_user_representation))
split_indices=number_of_maj_users

In [ ]:
#If plotted without respect to the subgroups, preference means look more or less normally distributed
plt.hist(actual_user_representation.mean(axis=1))



In [ ]:
#when plotting out mean preferences when accounting for group, we can see a clear distinction in preference
plt.hist(maj_user_representation.mean(axis=1), alpha=.7, color='b')
plt.hist(min_user_representation.mean(axis=1), alpha=0.7, color='r')



In [ ]:
filtering = ContentFiltering(actual_user_representation=actual_user_representation, 
                             num_attributes=number_of_attributes,
                             num_items=500)


mse = MSEMeasurement(diagnostics=True)
recall=RecallMeasurement()

filtering.add_metrics(mse, recall)

filtering.startup_and_train(50)
filtering.run(450)

In [ ]:
mse_diagnostics = filtering.metrics[0].get_diagnostics()
mse_beginning = mse_diagnostics.loc[50:, :]
mse_beginning.head()



In [ ]:
def mse_histogram(model, split_indices=None):
    metric = (
                model.predicted_scores.value.mean(axis=1)- model.users.actual_user_scores.value.mean(axis=1))** 2
    
    colors = ["blue", "orange", "red", "yellow", "green"]
    if split_indices is not None and len(split_indices) > 0:
        splits = [0] + split_indices + [metric.size]
        for i in range(len(splits) - 1):
            values = metric[splits[i] : splits[i + 1]]
            plt.hist(values, alpha=0.7, color=colors[i])
    else:
        plt.hist(metric, bins="auto")
        plt.ylabel("observation count (total n={})".format(metric.size))
        plt.xlabel("mean sqaured error")

In [ ]:
print(in_k)
print(not_in_k)
print(len(model.interactions))
k

In [ ]:

np.concatenate((np.ones(len(in_k)), np.zeros(len(not_in_k))), axis=None)

In [ ]:
colors = ["blue", "orange", "red", "yellow", "green"]
split_indices = number_of_maj_users
shown_item_scores = np.take(model.predicted_scores.value, model.items_shown)
shown_item_ranks = np.argsort(shown_item_scores, axis=1)
top_k_items = np.take(model.items_shown, shown_item_ranks[:, k :])
in_k = (np.where(np.isin(model.interactions, top_k_items))[0])
not_in_k = (np.where(~np.isin(model.interactions, top_k_items))[0])
metric = np.concatenate((np.ones(len(in_k)), np.zeros(len(not_in_k))), axis=None)

maj_population_outcomes = metric[:split_indices]
min_population_outcomes = metric[split_indices:]

plt.hist(maj_population_outcomes, color=colors[0])
plt.hist(min_population_outcomes, color=colors[1])



# if split_indices is not None:
#     splits = [0] + split_indices + [metric.size]
#     for i in range(len(splits) - 1):
#         values = metric[splits[i] : splits[i + 1]]
#         plt.hist(values, alpha=0.7, color=colors[i])

# plt.hist(metric, bins="auto")
# plt.ylabel("observation count (total n={})".format(metric.size))
# plt.xlabel("recall at k")


# 
#     if split_indices is not None and len(split_indices) > 0:
#         splits = [0] + split_indices + [metric.size]
#         for i in range(len(splits) - 1):
#             values = metric[splits[i] : splits[i + 1]]
#             plt.hist(values, alpha=0.7, color=colors[i])
#     else:
#         plt.hist(metric, bins="auto")
#         plt.ylabel("observation count (total n={})".format(metric.size))
#         plt.xlabel("mean sqaured error")

In [ ]:
plt.hist(min_population_outcomes)

In [ ]:
len(min_population_outcomes)

In [ ]:
metric_histogram(filtering)

In [ ]:
metric_histogram(filtering, [number_of_maj_users])

In [ ]:
#generate a bimodal distribution
N=500
mu, sigma = 1.845, 1
mu2, sigma2 = 5.845, 1
X1 = np.random.normal(mu, sigma, N)
X2 = np.random.normal(mu2, sigma2, N)
X_bimodal = np.concatenate([X1, X2])

In [ ]:
# print majority / minority outcome stats
def majority_minority_outcomes(metric, split_index):
    split_indices = [split_index]

        
    maj_mean = metric.last_observation[:split_index].mean()
    maj_std = metric.last_observation[:split_index].std()

    min_mean = metric.last_observation[split_index:].mean()
    min_std = metric.last_observation[split_index:].std()

    print("Majority group statistics: ", maj_mean, "(mean), ", maj_std, "(std)")
    print("Minority group statistics: ", min_mean, "(mean), ", min_std, "(std)")
    print()
    
    metric.hist(split_indices)

In what follows, we expand on this minimal example to gain a deeper understanding of what happens under the hood.

In [ ]:
filtering = ContentFiltering(actual_user_representation=actual_user_representation, 
                             num_attributes=number_of_attributes,
                             num_items=500)


mse = MSEMeasurement(diagnostics=True)
filtering.add_metrics(mse)

filtering.startup_and_train(50)
majority_minority_outcomes(mse, number_of_maj_users)


In [ ]:
filtering.run(450)
majority_minority_outcomes(mse, number_of_maj_users)

In [ ]:
bimodal = plt.hist(X_bimodal, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Bimodal Distribution')
plt.show()

In [ ]:
print(np.mean(X_bimodal))
print(np.std(X_bimodal))

In [ ]:
N=1000
mu, sigma = 14.99, 4
X1 = np.random.normal(mu, sigma, N)
X_skew = np.log2(X1)

skew = plt.hist(X_skew, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Skewed Distribution')
plt.show()

In [ ]:
N=1000
mu, sigma = 3.85, 2.2
normal_dist = np.random.normal(mu, sigma, N)

skew = plt.hist(normal_dist, bins=30)
plt.xlabel('Dependent Variable Value')
plt.ylabel('Number of Observations')
plt.title('Normal Distribution')
plt.show()

In [ ]:
#How to characterize power, type 1 vs type 2 errors 